# **TR-2022/11 - Technical report: Piracambaia II, Campinas, Brazil Groundwater Hydrochemical Dataset 2019**

|Technical Report ID  |2022/11|
|--|--|
| Title |Construction of Study Dataset|
| Authors | Diego Xavier Machado, Paula Dornhofer, Ana Elisa Abreu|
| Creation Date| 2023 - 04|


# Descrição Geral

Este notebook apresenta as etapas de formatação e filtragem dos dados coletados pelo Instituto de Geociências da Unicamp, além de gerar tabelas de estatística descritiva e gráficos de boxplot e de correlação referentes a esses dados.

Os dados que compõem a base de dados de estudo correspondem a amostras de água coletadas em dois períodos sazonais distintos: um período de chuvas (abril de 2019) e um período de seca (agosto de 2019). Embora o conjunto de dados original contivesse  20 observações para cada período, ao analisar-se todos os resultados laboratoriais, observou-se que a amostra de abril de 2019, referente ao poço identificado como ``w45'', continha dados faltantes. Por meio de discussões realizadas com a coorientadora deste trabalho, a especialista em hidrogeologia Profa. Dra. Ana Elisa Silva de Abreu, decidiu-se que tal amostra poderia ser descartada. Assim, foram mantidos os dados de 19 amostras referentes ao período de abril de 2019 e 20 amostras associadas  ao período de agosto de 2019. Tal informação é relevante, pela intenção de se realizar análises individualizadas para os períodos de chuva e seca.

Ao final da etapa de concatenação informações oriundas de diferentes documentos e planilhas, a base de dados  estudos resultante é caracterizada por uma planilha com um total de 39 linhas e 49 colunas, onde:
* A primeira coluna (coluna 0) identifica o ponto de coleta da amostra de água;
* A coluna 1 contém a data da coleta das amostras;
* As colunas 2 até 33 contêm as variáveis dos cátions de cada amostra coletada;
* As colunas 34 até 38 contêm as variáveis dos ânions de cada amostra coletada;
* A coluna 39 contêm o Carbono Orgânico Dissolvido em cada amostra coletada;
* As colunas 40 até 43 contêm as variáveis dos aspectos físico-químicos das amostras coletadas;
* As colunas 44 até 46 contêm as variáveis da análise isotópica das amostras coletadas;
* As colunas 47 e 48 contêm as coordenadas geográficas (no formato UTM) de cada ponto de coleta das águas.


Ademais, as amostras foram nomeadas como nos documentos utilizados para formar a base de dados. Os poços domésticos foram nomeados como "WX", onde X é a enumeração dada pelos pesquisadores que fizeram as coletas. O ponto de coleta das águas superficiais do rio foi chamada de *river*. Por fim, o ponto de coleta das águas superficiais do meandro foi nomeado de *meander*.

A princípio buscou-se padronizar a tabela de entrada, com todos os dados das coletas hidroquímicas concatenados em apenas uma tabela, sendo a primeira coluna o local onde foram coletadas as amostras (coluna "*well*"), a segunda coluna sendo a data no formato MM/AAAA (coluna "*date*") e as demais colunas sendo as variáveis dos dados, no qual desejamos aplicar os algoritmos de aprendizado de máquina. Juntamente com a tabela de entrada, é utilizado em conjunto uma tabela com os limites de detecção do aparelho utilizado para cada variável mensurada. Ambas tabelas são salvas em um aquivo do tipo `.csv`.

A partir da tabela de entrada, podemos seguir com os próximos passos de tratamento da tabela, que será transformado em um objeto do tipo pandas.DataFrame.

In [ ]:
#-----------------------------------------------------#
#              IMPORTAÇÃO DAS BIBIOTECAS              #
#-----------------------------------------------------#

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns

# Arquivos Base

A partir desses dados foram criadas as planilhas CSV:
* Na planilha `tabela_coletas.csv`, temos a tabela principal de entrada padronizada como detalhado anteriormente.
* Na planilha `limites_deteccao.csv`, temos separado os limites de detecção do aparelho que foi utilizado


In [ ]:
#-----------------------------------------------------#
#               IMPORTAÇÃO DOS ARQUIVOS               #
#-----------------------------------------------------#

from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv('/content/drive/Shareddrives/datasci4water/IG/data/interim/tabela_coletas.csv')
df = df.reset_index(drop=True)

limites_deteccao = pd.read_csv('/content/drive/Shareddrives/datasci4water/IG/data/interim/limites_deteccao.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Substituição dos valores "<LD" e "<LQ"

* LD : Limite de detecção
* LQ: Limite de quantificação

Nessa etapa, conforme indicado pela professora coorientadora do projeto, Profa. Dra. Ana Elisa Silva de Abreu, do IG, caso o valor mensurado pelo aparelho entregue um valor menor que o LD, esse valor será substituído por um valor 2 vezes maior que esse limite.

In [ ]:
#-----------------------------------------------------#
#          SUBSTITUIÇÃO DE VALORES INVÁLIDOS          #
#-----------------------------------------------------#

for i in range(df.shape[0]):
  for j in range(limites_deteccao.iloc[:, 0].values.size):
    if df.iloc[i, j+2] == '<LQ' or df.iloc[i, j+2] == '<LD':
      df.iloc[i, j+2] = 2*np.float64(limites_deteccao.iloc[j, 1])

for j in range(2, df.shape[1]):
  for i in range(df.shape[0]):
    df.iloc[i, j] = np.float64(df.iloc[i, j])
  df[df.columns[j]] = df[df.columns[j]].astype(np.float64)

# Remover os componentes químicos no qual possuem a mediana menor que o LQ
Devido a algumas substâncias químicas possuírem um valor menor que os limites de detecção ou quantificação para várias substâncias, utilizaremos o parâmetro da mediana para ajudar a filtrar a tabela, removendo variáveis que poderiam causar ruídos ou imprecisões ao implementar os algoritmos de aprendizado de máquina.

O processo de filtragem dos valores é baseado em remover as variáveis cujo a mediana possui um valor menor do que os limites de quantificação do aparelho utilizado para mensurar as substâncias químicas.

In [ ]:
#-----------------------------------------------------#
#            PRIMEIRA FILTRAGEM DOS DADOS             #
#-----------------------------------------------------#

medianas = (df.describe()).iloc[5, :]

para_remover = []
for i in range(limites_deteccao.shape[0]):
  if float(medianas.iloc[i]) < 4*float(limites_deteccao.iloc[i, 1]):
    para_remover.append(limites_deteccao.iloc[i, 0])


In [ ]:
limites_deteccao

,Elemento,Limite
0,Li,0.0100
1,Be,0.0070
2,B,1.2000
3,Na,2.0000
4,Mg,0.2000
...,...,...
59,Cl,0.0025
60,NO2,0.0050
61,NO3,0.0100
62,SO4,0.0050


### Impressão dos boxplots dos dados removidos

In [ ]:
print(para_remover)

['Be', 'Sc', 'Ti', 'Se', 'Zr', 'Nb', 'Ag', 'Cd', 'Sn', 'Hf', 'Ta', 'W', 'Bi', 'Th', 'NH4', 'NO2']


In [ ]:
list((limites_deteccao.loc[limites_deteccao['Elemento'] == 'Be']).values)[0][1]

0.007

In [ ]:
#-----------------------------------------------------#
#                      BOXPLOTS                       #
#-----------------------------------------------------#

for coluna in para_remover:
  fig = px.box(df, y=df[coluna], width=500, height=500, hover_data=[df['well'], df['date']])
  fig.update_layout(font=dict(size=15))


  fig.layout.xaxis2 = go.layout.XAxis(overlaying='x', range=[0, 2], showticklabels=False)

  limite = list((limites_deteccao.loc[limites_deteccao['Elemento'] == 'Be']).values)[0][1]

  fig.add_scatter(x = [0, 2], y = [limite, limite], mode='lines', xaxis='x2',
                              showlegend=True, name='LD: ' + str(limite), line=dict(dash='dash', color = "firebrick", width = 2))
  fig.layout.xaxis2 = go.layout.XAxis(overlaying='x', range=[0, 2], showticklabels=False)

  fig.add_scatter(x = [0, 2], y = [4*limite, 4*limite], mode='lines', xaxis='x2',
                              showlegend=True, name='LQ: ' + str(4*limite), line=dict(dash='dash', color = "green", width = 2))
  fig.show()

In [ ]:
df_filtrada = df.drop(columns=para_remover)

# Fazendo a estatística descritiva e a matriz de correlação
Como agora temos uma tabela já filtrada, podemos fazer estatística descritiva e criando uma nova tabela com os seguintes dados:
* Contagem
* Média
* Desvio Padrão
* Mínimo
* Primeiro Quartil
* Segundo Quartil
* Terceiro Quartil
* Máximo

Além disso, é feita a matriz de correlação dos dados, para a análise de outras possíveis variáveis para remover e diminuir a dimensão da tabela.

In [ ]:
#-----------------------------------------------------#
#               ESTATÍSTICA DESCRITIVA                #
#-----------------------------------------------------#

df_descritiva = df_filtrada.describe()

novo_limite = limites_deteccao.set_index('Elemento').T # Com a remoção de algumas substâncias químicas, uma nova tabela de limites de detecção é feita, removendo da original as variáveis descartadas
novo_limite = novo_limite.drop(columns=para_remover)

df_correlacoes = df_filtrada.corr()

# correlacoes = df_filtrada.corr().where(np.triu(np.ones(df_filtrada.corr().shape)).astype(bool)).unstack().reset_index()
# correlacoes.columns = ['variavel1', 'variavel2', 'correlacao']
# correlacoes = correlacoes[(correlacoes['correlacao'] != 1.0)]

<ipython-input-31-3e140d26c307>:10: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_correlacoes = df_filtrada.corr()


In [ ]:
#-----------------------------------------------------#
#                  SALVANDO OS DADOS                  #
#-----------------------------------------------------#

df_filtrada.to_csv('/content/drive/Shareddrives/datasci4water/IG/data/interim/df_filtrada.csv', index=False)
df_descritiva.to_csv('/content/drive/Shareddrives/datasci4water/IG/data/interim/estatistica_descritiva.csv')
df_correlacoes.to_csv('/content/drive/Shareddrives/datasci4water/IG/data/interim/matriz_correlacao.csv', index=False)
novo_limite.to_csv('/content/drive/Shareddrives/datasci4water/IG/data/interim/limites_filtrados.csv', index=False)

# Imprimindo o primeiro *Heatmap*
Com a matriz de correlação gerada, criamos um *Heatmap* para que a professora cooerientadora possa analizar a correlação entre as variáveis e indicá-las para remover da tabela final.

In [ ]:
#-----------------------------------------------------#
#                   PRIMEIRO HEATMAP                  #
#-----------------------------------------------------#

# Cria um heatmap dinâmico usando o plotly
fig = px.imshow(df_correlacoes.values,
                x=df_correlacoes.columns,
                y=df_correlacoes.columns,
                color_continuous_scale='RdBu',
                zmin=-1,
                zmax=1,
                labels=dict(color="Correlação"))

# Adiciona recursos interativos
fig.update_layout(
    xaxis=dict(side='top'),
    width=1200,
    height=1200,
    margin=dict(l=50, r=50, b=100, t=100, pad=4),
    title="Mapa de calor da matriz de correlação"
)

fig.show()

fig.write_html('/content/drive/Shareddrives/datasci4water/IG/data/interim/heatmap1.html') # Salvamento do heatmap

# Imprimindo *Boxplots*
Nessa etapa ocorre a impressão do *boxplot* de cada variável. Para cada boxplot referente a uma substância química (no qual há um limite de detecção), há como referência linhas horizontais que representam os limites de detecção (LD) e os limites de quantificação (LQ).

In [ ]:
#-----------------------------------------------------#
#                      BOXPLOTS                       #
#-----------------------------------------------------#

for i in range(2,  novo_limite.size+2):
  fig = px.box(df_filtrada, y=df_filtrada.columns[i], width=500, height=500, hover_data=[df_filtrada['well'], df_filtrada['date']])
  fig.update_layout(font=dict(size=15))

  # fig.layout.xaxis2 = go.layout.XAxis(overlaying='x', range=[0, 2], showticklabels=False)
  # fig.add_scatter(x = [0, 2], y = [20, 20], mode='lines', xaxis='x2', showlegend=False, line=dict(dash='dash', color = "firebrick", width = 2))

  fig.layout.xaxis2 = go.layout.XAxis(overlaying='x', range=[0, 2], showticklabels=False)
  fig.add_scatter(x = [0, 2], y = [novo_limite.iloc[0, i-2], novo_limite.iloc[0, i-2]], mode='lines', xaxis='x2',
                              showlegend=True, name='LD: ' + str(novo_limite.iloc[0, i-2]), line=dict(dash='dash', color = "firebrick", width = 2))
  fig.layout.xaxis2 = go.layout.XAxis(overlaying='x', range=[0, 2], showticklabels=False)
  # fig.add_scatter(x = [0, 2], y = [2*novo_limite.iloc[0, i-2], 2*novo_limite.iloc[0, i-2]], mode='lines', xaxis='x2',
  #                             showlegend=True, name='2*LD', line=dict(dash='dash', color = "yellow", width = 2))
  fig.add_scatter(x = [0, 2], y = [4*novo_limite.iloc[0, i-2], 4*novo_limite.iloc[0, i-2]], mode='lines', xaxis='x2',
                              showlegend=True, name='LQ: ' + str(4*novo_limite.iloc[0, i-2]), line=dict(dash='dash', color = "green", width = 2))
  fig.show()

for i in range(novo_limite.size +2, df_filtrada.columns.size):

  fig = px.box(df_filtrada, y=df_filtrada.columns[i], width=500, height=500, hover_data=[df_filtrada['well'], df_filtrada['date']])
  fig.update_layout(font=dict(size=15))
  fig.show()


# Removendo os elementos escolhidos
A professora coorientadora indicou a remoção das seguintes variáveis para a tabela final:
 - **La**, **Ce**, **Pr**, **Sm**, **Eu**, **Tb**, **Dy**, **Ho**, **Tm**, **Yb**, **Lu** e **TDS**.


Em seguida, temos o novo *Heatmap* formado após a remoção dessas variáveis.

In [ ]:
#-----------------------------------------------------#
#               REMOÇÃO DAS VARIÁVIES                 #
#-----------------------------------------------------#

para_remover = ['La', 'Ce', 'Pr', 'Sm', 'Eu', 'Tb', 'Dy', 'Ho', 'Tm', 'Yb', 'Lu', 'TDS']

df_final = df_filtrada.drop(columns=para_remover)

df_final.to_csv('/content/drive/Shareddrives/datasci4water/IG/data/interim/df_final.csv', index=False) # Salvamento da tabela final

In [ ]:
#-----------------------------------------------------#
#                   SEGUNDO HEATMAP                   #
#-----------------------------------------------------#

# Cria um heatmap dinâmico usando o plotly
fig = px.imshow(df_final.corr().values,
                x=df_final.corr().columns,
                y=df_final.corr().columns,
                color_continuous_scale='RdBu',
                zmin=-1,
                zmax=1,
                labels=dict(color="Correlação"))

# Adiciona recursos interativos
fig.update_layout(
    xaxis=dict(side='top'),
    width=1200,
    height=1200,
    margin=dict(l=50, r=50, b=100, t=100, pad=4),
    title="Mapa de calor da matriz de correlação"
)

fig.show()
fig.write_html('/content/drive/Shareddrives/datasci4water/IG/data/interim/heatmap2.html') # Salvamento do novo heatmap

<ipython-input-8-0820bcd504e2>:6: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  fig = px.imshow(df_final.corr().values,
<ipython-input-8-0820bcd504e2>:7: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  x=df_final.corr().columns,
<ipython-input-8-0820bcd504e2>:8: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  y=df_final.corr().columns,
